# Configuration

In [1]:
# Parameters
ENABLE_COLAB = False
PROJECT_NAME = 'ML1010-Group-Project'

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni//ML_Root/project_root'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

START_DATA_FILE = '01_Cellphone_full.pkl.gz'
END_DATA_FILE = '01_Cellphone_full_TFIDF.pkl.gz'

##This is for experiment work only###########################
#EXPERIMENT_NAME = 'ReviewText_Lemma_Bert (XGB)'
#FILE_NAME = '01_ML1010_GP_XGB_Bert'
#LOAD_FROM_EXP = False

# Bootstrap Environment

In [2]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)



import mv_python_utils as mvutils


Wha...where am I?
I am awake now.

I have set your current working directory to /home/magni/ML_Root/project_root/ML1010-Group-Project
The current time is 18:22
Hello sir. Reminder, no more coffee.



# Setup Runtime Environment

In [3]:
#%%capture
if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q 
  !pip install sentence_transformers -q
  !pip install flair -q
  !pip install spacy -U -q
  !pip install spacytextblob -q
  display('Google Colab enabled')
else:
  display('Google Colab not enabled')

#Common imports
#import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt

#!python -m spacy download en_core_web_sm

#nltk.download('stopwords')
#%matplotlib inline

'Google Colab not enabled'

# Load Data

In [4]:
### load the meta data
dataOrig = pd.read_pickle(jarvis.DATA_DIR_WORK + '/' + START_DATA_FILE)
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506438 entries, 0 to 506437
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   uuid                   506438 non-null  object        
 1   overall                506438 non-null  float64       
 2   verified               506438 non-null  bool          
 3   reviewTime             506438 non-null  datetime64[ns]
 4   reviewYear             506438 non-null  int64         
 5   reviewerID             506438 non-null  object        
 6   asin                   506438 non-null  object        
 7   style                  215400 non-null  object        
 8   reviewText             506438 non-null  object        
 9   summary                506438 non-null  object        
 10  vote                   103930 non-null  object        
 11  category               506438 non-null  string        
 12  description            506438 non-null  obje

# Column Explosion

In [5]:
pd.set_option('mode.chained_assignment', None)

In [6]:
import cw_df_metric_utils as cwutils
import importlib
import DataPackage as dp
import DataPackageSupport as dps
import DataExperiment
import DataExperimentSupport
import time

2022-01-25 18:23:22.066501: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-25 18:23:22.066542: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
importlib.reload(cwutils)
importlib.reload(dp)
importlib.reload(dps)
importlib.reload(DataExperiment)
importlib.reload(DataExperimentSupport)

In [7]:
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506438 entries, 0 to 506437
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   uuid                   506438 non-null  object        
 1   overall                506438 non-null  float64       
 2   verified               506438 non-null  bool          
 3   reviewTime             506438 non-null  datetime64[ns]
 4   reviewYear             506438 non-null  int64         
 5   reviewerID             506438 non-null  object        
 6   asin                   506438 non-null  object        
 7   style                  215400 non-null  object        
 8   reviewText             506438 non-null  object        
 9   summary                506438 non-null  object        
 10  vote                   103930 non-null  object        
 11  category               506438 non-null  string        
 12  description            506438 non-null  obje

# Create TF-IDF and save to file

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=768)
x = v.fit_transform(dataOrig['reviewText_lemma'])

In [9]:
feature_names = v.get_feature_names_out()
print(feature_names)

['00' '10' '100' '12' '15' '16' '20' '200' '30' '32' '40' '4s' '50' '64'
 '99' 'ability' 'able' 'absolutely' 'access' 'accessory' 'account'
 'activate' 'actually' 'adapter' 'add' 'address' 'advertise' 'ago' 'alarm'
 'allow' 'amazing' 'amazon' 'android' 'annoying' 'answer' 'anymore' 'app'
 'appear' 'apple' 'application' 'area' 'arrive' 'ask' 'asus' 'at' 'att'
 'audio' 'auto' 'automatically' 'available' 'average' 'away' 'awesome'
 'background' 'backup' 'bad' 'band' 'bar' 'barely' 'base' 'basic'
 'basically' 'battery' 'beat' 'beautiful' 'begin' 'believe' 'big' 'bit'
 'black' 'blackberry' 'bloatware' 'blu' 'blue' 'bluetooth' 'book' 'boost'
 'boot' 'bother' 'box' 'brand' 'break' 'bright' 'brightness' 'bring'
 'browse' 'browser' 'buck' 'budget' 'build' 'business' 'button' 'buy'
 'cable' 'calendar' 'call' 'camera' 'capability' 'car' 'card' 'care'
 'carrier' 'carry' 'case' 'cause' 'cell' 'cellphone' 'cellular' 'certain'
 'change' 'charge' 'charger' 'cheap' 'check' 'chinese' 'choice' 'choose'
 

In [10]:
#x.shape
tDf = pd.DataFrame(data=x.toarray(), columns = feature_names)
tDf.head(2)

,00,10,100,12,15,16,20,200,30,32,...,wonderful,word,work,world,worth,write,wrong,year,yes,youtube
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.095399,0.0,0.191165,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.095399,0.0,0.191165,0.0,0.0,0.0,0.0,0.0


In [11]:
tDfUn = dataOrig[['uuid', 'overall_posneg']].copy()
tDfUn.head(2)

,uuid,overall_posneg
0,7fd30754-ab43-442a-808c-9f3b30d3f6a3,0
1,1604cce1-7edd-4094-b32d-485d3563ad1f,0


In [12]:
dfTfidf = pd.concat([tDfUn, tDf], axis=1)
dfTfidf.head(2)

,uuid,overall_posneg,00,10,100,12,15,16,20,200,...,wonderful,word,work,world,worth,write,wrong,year,yes,youtube
0,7fd30754-ab43-442a-808c-9f3b30d3f6a3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.095399,0.0,0.191165,0.0,0.0,0.0,0.0,0.0
1,1604cce1-7edd-4094-b32d-485d3563ad1f,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.095399,0.0,0.191165,0.0,0.0,0.0,0.0,0.0


In [13]:
dfTfidf.to_pickle(jarvis.DATA_DIR_EXP + '/reviewText_TF-IDF_Full.pkl.gz')

# section below is adding tfidf as a column similar to BERT, GLOVE, MPNet
# problem is that you lose column names.

In [ ]:
#add encoding to frame
dataOrig['reviewText_lemma_TFIDF'] = x.toarray().tolist()

In [ ]:
dataOrig.head(2)

In [ ]:
#%%time
#cwutils.expandColumn(dataOrig, 'reviewText')
#cwutils.flairSentimentEncode(dataOrig, 'reviewText_lemma')
#cwutils.calcTextBlobSentiment(dataOrig, 'reviewText_lemma')
#cwutils.bertEncode(dataOrig, 'reviewText_lemma', )
#cwutils.splitSpacySentences(dataOrig, 'reviewText_lemma')

cwutils.columnEncode(data=dataOrig,
                   columnName='reviewText_lemma',
                   transformerType='sentence-transformers/distilbert-base-nli-mean-tokens',
                   colSuffix='_bert')

dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)


cwutils.columnEncode(data=dataOrig,
                   columnName='reviewText_lemma',
                   transformerType='sentence-transformers/average_word_embeddings_glove.6B.300d',
                   colSuffix='_glove')

dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)

cwutils.columnEncode(data=dataOrig,
                   columnName='reviewText_lemma',
                   transformerType='sentence-transformers/all-mpnet-base-v2',
                   colSuffix='_mpnet')

dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)


# Scratchpad

In [ ]:
dataOrig.info()

# Save Files!!!!!!!!

In [ ]:
#dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)